In [1]:
import pyvista as pv
from pyvista import examples
import nibabel as nib
import numpy as np

In [2]:
file_path = r"copd1_4D.nii.gz"
image_data = nib.load(file_path).get_fdata()
# Extraer datos de la fase inspiratoria
inspiratory_data = image_data[:, :, :, 0]  # Selecciona la primera fase (índice 0)
grid = pv.ImageData()
grid.dimensions = np.array(inspiratory_data.shape)
grid.origin = (1, 1, 1)
grid.spacing = (0.625, 0.625, 2.5)
grid.point_data["values"] = inspiratory_data.flatten(order="F")  # Flatten the array

In [2]:
file_path = r"copd1_4D.nii.gz"
image_data = nib.load(file_path).get_fdata()
# Extraer datos de la fase inspiratoria
inspiratory_data = image_data[:, :, :, 0]  # Selecciona la primera fase (índice 0)
grid = pv.ImageData()
grid.dimensions = np.array(inspiratory_data.shape)
grid.origin = (1, 1, 1)
grid.spacing = (0.625, 0.625, 2.5)
grid.point_data["values"] = inspiratory_data.flatten(order="F")  # Flatten the array
pl = pv.Plotter(shape=[1,4])
_ = pl.add_volume(grid,mapper = 'gpu')
pl.add_text("volumetric image")
pl.subplot(0, 1)
_ = pl.add_mesh(grid)
pl.add_text("object image")
pl.subplot(0, 2)
_ = pl.add_mesh_isovalue(grid)
pl.add_text("isovalue")
pl.subplot(0, 3)

isovalues = grid.contour(np.linspace(0, 2000, 10), method='contour') #'contour', 'marching_cubes' and 'flying_edges'
# Create a new lookup table with oranges
lut = pv.LookupTable()
lut.value_range = (0, 2000)
lut.hue_range = (0.0, 1)
lut.saturation_range = (0.0, 1)
lut.alpha_range = (0.0, 1.0)
lut.scalar_range = (1, 2000)

scalars_rng = (isovalues.active_scalars.min(), isovalues.active_scalars.max())


def make_double_slider(attr, idx):
    """Create a double slider for a given lookup table attribute."""

    def set_min(min_value):
        max_value = getattr(lut, attr)[1]
        if min_value > max_value:
            # force the movement of the maximum value
            max_value = min_value
            pl.slider_widgets[idx * 2 + 1].GetRepresentation().SetValue(max_value)
        setattr(lut, attr, (min_value, max_value))

        if attr == 'scalar_range':
            actor.mapper.scalar_range = getattr(lut, attr)

    def set_max(max_value):
        min_value = getattr(lut, attr)[0]
        if max_value < min_value:
            # force the movement of the minimum value
            min_value = max_value
            pl.slider_widgets[idx * 2].GetRepresentation().SetValue(min_value)
        setattr(lut, attr, (min_value, max_value))

        if attr == 'scalar_range':
            actor.mapper.scalar_range = getattr(lut, attr)

    if attr == 'scalar_range':
        rng = scalars_rng
    else:
        rng = (0, 1)

    # create two overlapping slider bars by hiding the tube of the second
    pl.add_slider_widget(
        set_min,
        rng,
        value=getattr(lut, attr)[0],
        interaction_event='always',
        title=' '.join(attr.split('_')).capitalize(),
        tube_width=0.003,
        pointa=(0.6, 0.9 - 0.165 * idx),
        pointb=(0.9, 0.9 - 0.165 * idx),
    )
    pl.add_slider_widget(
        set_max,
        rng,
        value=getattr(lut, attr)[1],
        interaction_event='always',
        tube_width=0.0,
        pointa=(0.6, 0.9 - 0.165 * idx),
        pointb=(0.9, 0.9 - 0.165 * idx),
    )
actor = pl.add_mesh(isovalues, cmap=lut, lighting=False)
make_double_slider('alpha_range', 0)
make_double_slider('hue_range', 1)
make_double_slider('value_range', 2)
make_double_slider('saturation_range', 3)
make_double_slider('scalar_range', 4)
pl.add_text("10 isovalues")

pl.link_views()
pl.show()

Widget(value='<iframe src="http://localhost:55514/index.html?ui=P_0x201d9236c90_0&reconnect=auto" class="pyvis…

In [8]:
pl = pv.Plotter()
_ = pl.add_volume(grid,mapper = 'gpu', opacity='sigmoid_10')
pl.show()

Widget(value='<iframe src="http://localhost:55514/index.html?ui=P_0x201d9aa1790_5&reconnect=auto" class="pyvis…

In [16]:
p = pv.Plotter()
p.add_mesh_threshold(grid)
p.show()

Widget(value='<iframe src="http://localhost:55514/index.html?ui=P_0x201f15ae010_11&reconnect=auto" class="pyvi…

In [4]:
grid = grid.contour(isosurfaces= [600], progress_bar= True, method='marching_cubes') #'contour', 'marching_cubes' and 'flying_edges'
pl = pv.Plotter()
actor = pl.add_mesh(grid)
widget = pl.add_affine_transform_widget(actor)
pl.show()

Computing Contour: 100%|██████████[00:00<00:00]


Widget(value='<iframe src="http://localhost:55905/index.html?ui=P_0x1ce8106ea50_1&reconnect=auto" class="pyvis…

In [6]:
# Download skybox
# cubemap = examples.download_sky_box_cube_map()
cubemap = examples.download_cubemap_space_4k()

pl = pv.Plotter()
pl.add_actor(cubemap.to_skybox())
pl.set_environment_texture(cubemap)  # For reflecting the environment off the mesh
pl.add_mesh(grid, color='ff9999', opacity = 0.5, ambient= 1,diffuse=1,specular= 1, lighting= True, roughness=1.0, metallic=0.0,pbr=True, interpolate_before_map=False, use_transparency=True, render = True)
pl.add_text("generic illumination scene")
pl.show()

Widget(value='<iframe src="http://localhost:55905/index.html?ui=P_0x1ce8c317990_3&reconnect=auto" class="pyvis…

In [7]:
# Función para crear sliders individuales

pl = pv.Plotter(lighting=None)
pl.set_background('black')
pl.add_mesh(grid, color= 'ff9999', opacity = 0, ambient= 1,diffuse=0,specular= 1, lighting= True, roughness=0.5, metallic=0.5,pbr=True, interpolate_before_map=True, use_transparency=True)
# set up lighting
light = pv.Light((-2, 2, 0), (0, 0, 0), 'white')
light.positional = True
light.cone_angle = 40
light.exponent = 10
light.intensity = 3
light.show_actor()
pl.add_light(light)

light = pv.Light((2, 0, 0), (0, 0, 0), (0.7, 0.0862, 0.0549))
pl.add_light(light)

light = pv.Light((0, 0, 10), (0, 0, 0), 'white')
pl.add_light(light)

light = pv.Light((0, 0, 10), (0, 0, 0), 'white')
pl.add_light(light)

pl.enable_shadows()
pl.show()

Widget(value='<iframe src="http://localhost:55905/index.html?ui=P_0x1ce902b74d0_4&reconnect=auto" class="pyvis…

In [26]:
grid.bounds

(1.0, 320.375, 66.30005645751953, 320.375, 1.0, 297.8765563964844)

In [ ]:
plotter = pv.Plotter(lighting='none', window_size=(1000, 1000))
plotter.add_mesh(grid, color='ff9999')
floor = pv.Plane(center=(*grid.center[:2], grid.bounds[-1]), i_size=1000, j_size=1000)
plotter.add_mesh(floor, color='green')

UFO = pv.Light(position=(160.6875, 193.33752822875977, 149.4382781982422 - 1000), focal_point=(0, 0, 0), color='white')
UFO.positional = True
UFO.cone_angle = 40
UFO.exponent = 10
UFO.intensity = 3
UFO.show_actor()
plotter.add_light(UFO)

# enable shadows to better demonstrate lighting
plotter.enable_shadows()
plotter.show_bounds()
# plotter.camera_position = [(28, 30, 22), (0.77, 0, -0.44), (0, 0, 1)]
plotter.show()

ERROR:root:1: #version 150
ERROR:root:0(168) : error C1503: undefined variable "vertexVC"
ERROR:root:Could not set shader program
ERROR:root:attempt to add attribute without a bound program for attribute vertexMC
ERROR:root:Error setting 'vertexMC' in shader VAO.
ERROR:root:1: #version 150
vtkShaderProgram (000001CFFCA2DCD0): 


ERROR:root:Could not set shader program
ERROR:root:attempt to add attribute without a bound program for attribute vertexMC
ERROR:root:Error setting 'vertexMC' in shader VAO.
ERROR:root:1: #version 150
vtkShaderProgram (000001CFFCA2DCD0): 


ERROR:root:Could not set shader program
ERROR:root:attempt to add attribute without a bound program for attribute vertexMC
ERROR:root:Error setting 'vertexMC' in shader VAO.
ERROR:root:1: #version 150
vtkShaderProgram (000001CFFCA2DCD0): 


ERROR:root:Could not set shader program
ERROR:root:attempt to add attribute without a bound program for attribute vertexMC
ERROR:root:Error setting 'vertexMC' in shader VAO.
ERROR:root:1:

Widget(value='<iframe src="http://localhost:55905/index.html?ui=P_0x1cf8c44be10_16&reconnect=auto" class="pyvi…

ERROR:root:1: #version 150
vtkShaderProgram (000001CFFCA2DCD0): 


ERROR:root:Could not set shader program
ERROR:root:attempt to add attribute without a bound program for attribute vertexMC
ERROR:root:Error setting 'vertexMC' in shader VAO.
ERROR:root:1: #version 150
vtkShaderProgram (000001CFFCA2DCD0): 


ERROR:root:Could not set shader program
ERROR:root:attempt to add attribute without a bound program for attribute vertexMC
ERROR:root:Error setting 'vertexMC' in shader VAO.
ERROR:root:1: #version 150
vtkShaderProgram (000001CFFCA2DCD0): 


ERROR:root:Could not set shader program
ERROR:root:attempt to add attribute without a bound program for attribute vertexMC
ERROR:root:Error setting 'vertexMC' in shader VAO.
ERROR:root:1: #version 150
vtkShaderProgram (000001CFFCA2DCD0): 


ERROR:root:Could not set shader program
ERROR:root:attempt to add attribute without a bound program for attribute vertexMC
ERROR:root:Error setting 'vertexMC' in shader VAO.
ERROR:root:1: #version 150
vtkShader